In [ ]:
import numpy as np

from matplotlib import pyplot as plt

from src.lib_phaseshifts import *
from src.lib_tensors import *
from src.lib_tscatf import *
from src.delta import delta_amplitude

In [ ]:
LMAX = 14  # maximum angular momentum to be used in calculation
n_beams = 9  # no. of TLEED output beams
n_atoms = 1  # currently 1 is the only possible choice
n_geo = 1  # number of geometric variations ('displacements') to be considered

# From Stdin
# DR0,DRPER,DRPAR: thermal vibration amplitude steps to be included in
# current variation - DR0 is always 0., DRPER = DRPAR forced

DR = 0.1908624

CDISP = np.full((n_geo, n_atoms, 3),dtype=np.float64,fill_value=np.nan)  # displaced positions of current atomic site for variation
for i in range(n_geo):
    CDISP[i][0][0] = -0.01*i + 0.05
    CDISP[i][0][1] = 0
    CDISP[i][0][2] = 0

# unit vectors in Angstrom
u_vec1 = np.array([1.2722, -2.2036])
u_vec2 = np.array([1.2722,  2.2036])

# area of (overlayer) lateral unit cell - in case TLEED wrt smaller unit cell is used, TVA from reference computation must be set.
unit_cell_area = np.linalg.norm(np.cross(u_vec1, u_vec2))
# In Bohr radii
unit_cell_area = unit_cell_area / BOHR**2

IEL = 1  # element no. (in phase shifts supplied with input) that delta amplitudes
#          will be calculated for (not necessarily the same element as the one
#          used in the reference calculation!) - IEL = 0 means a vacancy will be assumed


In [ ]:
def _select_phaseshifts(IEL, phaseshifts):
    """Selects the phaseshifts for the given element number IEL"""
    return jax.lax.select(IEL == 0,
                          jnp.zeros_like(phaseshifts[:, 0, :]),
                          phaseshifts[:, IEL-1, :])


In [ ]:
_, phaseshifts, _, _ = readPHASESHIFTS(None, None, readfile='PHASESHIFTS', check=False, ignoreEnRange=False)

n_energies = 0
with open('T_1', 'r') as datei:
    for zeile in datei:
        if '-1' in zeile:
            n_energies += 1

tensor_dict = read_tensor('T_1', n_beams=9, n_energies= n_energies, l_max=LMAX+1)

# TODO: raise Error if requested energies are out of range respective to
# phaseshift energies (can't interpolate if out of range)

energies = np.array([tensor_dict['e_kin'][i] for i in range(n_energies)])
interpolated_phaseshifts = interpolate_phaseshifts(phaseshifts, LMAX, energies)

In [ ]:
displacement = np.array(CDISP[0,...])
delta_amplitude(IEL, LMAX, DR, tensor_dict, unit_cell_area, interpolated_phaseshifts, displacement)